## Import Libraries

In [ ]:
import torch
from torchvision import transforms, models
import torch.optim as optim

import requests

from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

## Helper Function

### load image

In [ ]:
def load_image(filename, shape=None):
  image = Image.open(filename).convert('RGB')
  size = image.size if max(image.size)<= 400 else 400
  size=shape if shape!= None else size
  image_transform =  transforms.Compose([transforms.Resize(size), transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
  image = image_transform(image)[:3,:,:].unsqueeze(0)
  return image

### Convert image for display

In [ ]:
def im_convert(image):
  image = image.to('cpu').clone().detach()
  image = image.numpy().squeeze()
  image = image.transpose(1,2,0)
  image = image * np.array((0.229, 0.224, 0.225))+np.array((0.485, 0.456, 0.406))
  image = image.clip(0,1)
  return image

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Display Images

In [ ]:
data_dir='images/'

In [ ]:
content_image = load_image(f'{data_dir}Personal.jpg').to(device)
style_image = load_image(f'{data_dir}hockney.jpg', shape=content_image.shape[-2:]).to(device)
fig, (ax1, ax2)=plt.subplots(1,2, figsize=(6,3))
ax1.imshow(im_convert(content_image))
ax2.imshow(im_convert(style_image))
ax1.set_title('Content')
ax2.set_title('Style')

## Load VGG19 model

In [ ]:
model = models.vgg19(weights='DEFAULT').features # 'VGG19_Weights.DEFAULT
for params in model.parameters():
  params.requires_grad_(False)
model.to(device)

In [ ]:
def get_features(image, model):
  layers = {
      '0': 'conv1_1',
      '5': 'conv2_1',
      '10': 'conv3_1',
      '19': 'conv4_1',
      '21': 'conv4_2',  ## content representation
      '28': 'conv5_1'
  }

  features = {}
  x=image
  for name, layer in model._modules.items():
    x=layer(x)
    if name in layers:
      features[layers[name]] = x

  return features

def gram_matrix(tensor):
  b, d, h, w = tensor.size()
  tensor = tensor.view(b*d, h*w)

  return torch.mm(tensor, tensor.t())

In [ ]:
content_features=get_features(content_image, model)
style_features = get_features(style_image, model)

style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}
target = content_image.clone().requires_grad_(True).to(device)

## Setting Weight and Loss

In [ ]:
style_weights= {
    'conv1_1': 1.0,
    'conv2_1': 0.75,
    'conv3_1': 0.6,
    'conv4_1': 0.2,
    'conv5_1': 0.2
}

content_weight = 1 #alpha
style_weight = 1e6 #beta

## Updating and Calculating Loss

In [ ]:
show_every = 400
optimizer = optim.Adam([target], lr=0.0005)

steps = 40000

for i in range(1, steps+1):
  target_features = get_features(target, model)
  content_loss = torch.mean((target_features['conv4_2']- content_features['conv4_2'])**2)

  style_loss = 0

  for layer in style_weights:
    target_feature = target_features[layer]
    _, d, h,w = target_feature.shape

    target_gram = gram_matrix(target_feature)

    style_gram = style_grams[layer]

    layer_style_loss = style_weights[layer]*torch.mean((target_gram - style_gram)**2)

    style_loss += layer_style_loss / (d*h*w)
  total_loss = content_weight*content_loss + style_weight*style_loss

  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if i% show_every==0:
    print(f'Total loss: ${total_loss.item()}')
    plt.imshow(im_convert(target))
    plt.title(f'Total loss: ${total_loss.item()}')
    plt.show()

![mixed_image.png](mixed_image.png)